In [1]:
!pip install -q rouge_score bert_score

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, TrainingArguments, pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_metric, Dataset
from tqdm import tqdm
from bert_score import score

2024-04-24 17:22:08.710120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 17:22:08.710248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 17:22:08.849128: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART").to('cuda')

tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
train_data = pd.read_csv("/kaggle/input/hindi-dataset/hindi_train.csv")
train_data.drop(columns=["Summary"], inplace=True)

test_data = pd.read_csv("/kaggle/input/hindi-dataset/HindiNews_test.csv")
test_data.rename(columns={"id": "Id"}, inplace=True)
data = pd.concat([train_data, test_data], axis=0)

train_dataset, test_dataset = train_test_split(data, test_size=0.15, shuffle=False)
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, shuffle=False)

In [9]:
def preprocess_data(example):
    inputs = tokenizer(example["Article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt").to('cuda')
    outputs = tokenizer(example["Heading"], padding="max_length", truncation=True, max_length=64, return_tensors="pt").to('cuda')
    return {"input_ids": inputs["input_ids"].tolist(), "attention_mask": inputs["attention_mask"].tolist(), "labels": outputs["input_ids"].tolist()}

train_dataset = Dataset.from_pandas(train_dataset)
train_dataset = train_dataset.map(preprocess_data, batched=True, batch_size=96)
val_dataset = Dataset.from_pandas(val_dataset)
val_dataset = val_dataset.map(preprocess_data, batched=True, batch_size=96)
test_dataset = Dataset.from_pandas(test_dataset)
test_dataset = test_dataset.map(preprocess_data, batched=True, batch_size=96)

Map:   0%|          | 0/18531 [00:00<?, ? examples/s]

Map:   0%|          | 0/2060 [00:00<?, ? examples/s]

Map:   0%|          | 0/3634 [00:00<?, ? examples/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/model",
    num_train_epochs=5,
    logging_dir="/kaggle/working/logs",
    logging_steps=500,
    overwrite_output_dir=True,
    save_steps=1000,
    eval_steps=500,
    save_total_limit=3,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

model.save_pretrained("/kaggle/working/finetuned_model")
tokenizer.save_pretrained("/kaggle/working/finetuned_model")

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,2.484400
1000,1.051800
1500,0.956300
2000,0.916400
2500,0.884500
3000,0.857100
3500,0.857500
4000,0.843700
4500,0.831800
5000,0.806700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/finetuned_model").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/finetuned_model", do_lower_case=False, use_fast=False, keep_accents=True)

def generate_heading(article):
    inputs = tokenizer(article, padding="max_length", truncation=True, max_length=512, return_tensors="pt").to('cuda')
    output = model.generate(inputs["input_ids"], max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

predictions = [generate_heading(article) for article in test_dataset["Article"]]

metric_rouge = load_metric("rouge")
references = test_dataset["Heading"]
rouge_scores = metric_rouge.compute(predictions=predictions, references=references)

P, R, F1 = score(predictions, references, lang='hi', verbose=True)

In [ ]:
with open("rouge_scores.txt", "w") as f:
    f.write(str(rouge_scores))

with open("bert_scores.txt", "w") as f:
    f.write(f"P: {P.mean()}\nR: {R.mean()}\nF1: {F1.mean()}")

In [ ]:
print(rouge_scores)

In [ ]:
print(F1.mean())